# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier


# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [22]:
df = pd.read_csv('previsao_de_renda.csv', index_col=0)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          12427 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usa

Abaio iremos excluir algumas colunas não relevantes no momento.

In [24]:
#excluindo id_cliente:
df=df.drop('id_cliente', axis=1)

#transformando a data para datetime:
df['data_ref'] = pd.to_datetime(df['data_ref'])

#criando uma nova coluna na qual somente terá a extração do mês da data_ref:
df['mes'] = df['data_ref'].dt.month

#excluindo a data_ref e deixando apenas o mês para facilitar a análise:
df=df.drop('data_ref', axis=1)

df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda,mes
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34,1
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15,1
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89,1
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77,1
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97,1


Agora vamos conferir se há valores faltantes:

In [25]:
df.isna().sum()

sexo                        0
posse_de_veiculo            0
posse_de_imovel             0
qtd_filhos                  0
tipo_renda                  0
educacao                    0
estado_civil                0
tipo_residencia             0
idade                       0
tempo_emprego            2573
qt_pessoas_residencia       0
renda                       0
mes                         0
dtype: int64

In [26]:
tipo_renda_frequente = df[df['tempo_emprego'].isnull()]['tipo_renda'].value_counts()
tipo_renda_frequente

Pensionista    2573
Name: tipo_renda, dtype: int64

Com a resposta acima podemos peceber que todos os valores faltantes são de pensionitas, o que explica não terem preenchido o campo, pois provavelmente não estão empregados, portanto, podemos preencher as células faltantes com 0.

In [51]:
df['tempo_emprego'] = df['tempo_emprego'].fillna(0)

In [14]:
df.isna().sum()

sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
mes                      0
dtype: int64

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [52]:
#separando x e y, aonde y recebe 'renda' e X todas as demais. 
X= df.drop('renda', axis=1)
y=df['renda']

In [53]:
#1-Separe a base em treinamento e teste (25% para teste, 75% para treinamento)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100) #25% separada para teste, logo 75% para teino

In [54]:
#transformando y_teste para series para que seja possível concatená-lo com o x_teste.
y_test=pd.Series(y_test)

In [55]:
y_train=pd.Series(y_train)

In [56]:
#concatenando x e y teste em uma única variável para podermos modelá-los #75%
treino=pd.concat([X_train, y_train], axis=1)

In [57]:
#concatenando x e y teste em uma única variável para podermos modelá-los. 25%
teste=pd.concat([X_test, y_test],axis=1)

In [58]:
# Valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Definição do modelo
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil, Treatment(0)) + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia'
md = smf.ols(modelo, data=treino)

# Lista para armazenar os resultados
resultados = []

# for para ajustar o modelo com os diferentes alphas
for alpha in alphas:
    reg = md.fit_regularized(method='elastic_net',
                             refit=True,
                             L1_wt=0.001, #valor próximo do zero para que pese mais em ridge
                             alpha=alpha)
    resultados.append(reg)

# Imprimir os resumos dos resultados para cada valor de alpha
for alpha, result in zip(alphas, resultados):
    print(f"Alpha = {alpha}")
    print(result.summary())
    print("\n")


Alpha = 0
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     159.2
Date:                Fri, 25 Aug 2023   Prob (F-statistic):               0.00
Time:                        06:54:28   Log-Likelihood:            -1.1591e+05
No. Observations:               11250   AIC:                         2.319e+05
Df Residuals:                   11225   BIC:                         2.321e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [ ]:
# Valores de alpha
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Definição do modelo
modelo = 'renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil, Treatment(0)) + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia'
md = smf.ols(modelo, data=treino)

# Lista para armazenar os resultados
resultados = []

# for para ajustar o modelo com os diferentes alphas
for alpha in alphas:
    reg = md.fit_regularized(method='elastic_net',
                             refit=True,
                             L1_wt=1, #peso em lasso
                             alpha=alpha)
    resultados.append(reg)

# Imprimir os resumos dos resultados para cada valor de alpha
for alpha, result in zip(alphas, resultados):
    print(f"Alpha = {alpha}")
    print(result.summary())
    print("\n")


As duas regularizações aplicadas acima, mesmo com alphas diferentes, nos entregaram resultados iguais, isso pode se dar por conta dos alphas testados serem bem próximos. Entretanto, nenhum dos modelos nos entregou somente variáveis significativas, portanto, acredito que há outras abordagens mais efetivas para seleção das variáveis. 

4-Rode um modelo stepwise. Avalie o  𝑅2
  na base de testes. Qual o melhor resultado?

Inicialmente, para rodarmos esse modelo, precisamos obter as dummies das variáveis object.

In [ ]:
#obtendo as dummies do teste
teste_dummies=teste
teste_dummies = pd.get_dummies(teste_dummies, columns=['sexo'], drop_first=True)
teste_dummies = pd.get_dummies(teste_dummies, columns=['posse_de_veiculo'], drop_first=True)
teste_dummies = pd.get_dummies(teste_dummies, columns=['posse_de_imovel'], drop_first=True)
teste_dummies = pd.get_dummies(teste_dummies, columns=['tipo_renda'], prefix='tipo', drop_first=True)
teste_dummies = pd.get_dummies(teste_dummies, columns=['estado_civil'], prefix='estado', drop_first=True)
teste_dummies = pd.get_dummies(teste_dummies, columns=['educacao'], prefix='grau', drop_first=True)
teste_dummies = pd.get_dummies(teste_dummies, columns=['tipo_residencia'], prefix='residencia', drop_first=True)

In [ ]:
#obtendo as dummies do treino
treino_dummies=treino
treino_dummies = pd.get_dummies(treino_dummies, columns=['sexo'], drop_first=True)
treino_dummies = pd.get_dummies(treino_dummies, columns=['posse_de_veiculo'], drop_first=True)
treino_dummies = pd.get_dummies(treino_dummies, columns=['posse_de_imovel'], drop_first=True)
treino_dummies = pd.get_dummies(treino_dummies, columns=['tipo_renda'], prefix='tipo', drop_first=True)
treino_dummies = pd.get_dummies(treino_dummies, columns=['estado_civil'], prefix='estado', drop_first=True)
treino_dummies = pd.get_dummies(treino_dummies, columns=['educacao'], prefix='grau', drop_first=True)
treino_dummies = pd.get_dummies(treino_dummies, columns=['tipo_residencia'], prefix='residencia', drop_first=True)

In [ ]:
#renomeando senão ele não roda no modelo abaixo.
teste_dummies['grau_Superior_completo']=teste_dummies['grau_Superior completo']
teste_dummies.drop('grau_Superior completo', axis=1)

#renomeando senão ele não roda no modelo abaixo.
treino_dummies['grau_Superior_completo']=treino_dummies['grau_Superior completo']
treino_dummies.drop('grau_Superior completo', axis=1)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


X = treino_dummies.drop('renda', axis=1)
y = treino_dummies['renda']

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out=0.05, 
                       verbose=True):
    included = list(initial_list)
    
    while True:
        changed = False
        
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=np.float64)
        
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        
        best_pval = new_pval.min()
        
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # Backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        
        if not changed:
            break
            
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')

In [ ]:
#utilizando as sugestões dadas pelo stepwise:
reg2=smf.ols('(renda)~ sexo_M + grau_Superior_completo + tempo_emprego + sexo_M + idade + tipo_Pensionista + tipo_Empresário + qt_pessoas_residencia + estado_União', data = teste_dummies).fit()

reg1.summary() #resumo da regressão

A adição de variáveis sugerida pelo stepwise trouxe um resultado melhor que a regularização ridge ou lasso, mas ainda podemos tentar algumas mudanças, como a transformação da variável renda. 

In [ ]:
reg2=smf.ols('np.log(renda)~ sexo_M + grau_Superior_completo + tempo_emprego + sexo_M + idade + tipo_Pensionista + tipo_Empresário + residencia_Estúdio + posse_de_imovel_True', data = teste_dummies).fit()

reg2.summary() #resumo da regressão

Esse segundo modelo explica melhor a variabilidade dos dados, tanto em comparação com o modelo anterior quanto com os modelos regularizados. Também tentou-se transformar renda para polinômio mas o R-quadrado ficou extremamente baixo, portanto ficaremos com o modelo em que transformamos a renda em log, que nos trouxe um R-quadrado de quase 35% e apenas variáveis significantes.

7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [ ]:
X_train=treino_dummies.drop('renda', axis=1)
y_train=treino_dummies['renda']
X_test=teste_dummies.drop('renda', axis=1)
y_test=teste_dummies['renda']

In [ ]:
# Fit regression model
#vamos rodar 2 arvores:
arvore_1 = DecisionTreeRegressor(max_depth=2) #profundidade 2
arvore_2 = DecisionTreeRegressor(max_depth=3) #profundidade 3

#treinar as duas com método fit:
arvore_1.fit(X_train, y_train)
arvore_2.fit(X_train, y_train)

In [ ]:
#o método score retorna o coeficiente de determinação da árvore
mse1 = arvore_1.score(X_train, y_train)
mse2 = arvore_2.score(X_train,y_train)

template = "O R-quadrado da árvore com profundidade={0} é: {1:.2f}"

print(template.format(arvore_1.get_depth(),mse1).replace(".",","))
print(template.format(arvore_2.get_depth(),mse2).replace(".",","))

A árvore com profundidade 2 nos traz um R-quadrado parecido com as regularizações lasso e ridge, enquanto que a árvore com profundidade 3 nos traz um R-quadrado maior que o modelo em que usamos as variáveis do stepwise e transformamos a renda com o log. Portanto, faremos a seguir a validação cruzada tanto para a árvore 1 quanto para a árvore 2, a fim de obter o erro médio quadrático e o desvio padrão, que nos permitirão avaliar qual árvore traz um resultado melhor. 

In [ ]:
#arvore 1 validação cruzada
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

# Realizando validação cruzada com 5 folds e calculando o negativo do MSE
mse_scores = cross_val_score(arvore_1, X, y, cv=5, scoring='neg_mean_squared_error')

# Convertendo os valores negativos de MSE para valores positivos
mse_scores = -mse_scores

# Calculando a média e o desvio padrão dos MSE
average_mse = mse_scores.mean()
std_mse = mse_scores.std()

print("MSE médio:", average_mse)
print("Desvio padrão do MSE:", std_mse)


In [ ]:
#arvore 2 validação cruzada
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

# Realizando validação cruzada com 5 folds e calculando o negativo do MSE
mse_scores = cross_val_score(arvore_2, X, y, cv=5, scoring='neg_mean_squared_error')

# Convertendo os valores negativos de MSE para valores positivos
mse_scores = -mse_scores

# Calculando a média e o desvio padrão dos MSE
average_mse = mse_scores.mean()
std_mse = mse_scores.std()

print("MSE médio:", average_mse)
print("Desvio padrão do MSE:", std_mse)


De acordo com as validações cruzadas, temos como preferível a árvore_2, visto que ela apresenta um desvio padrão menor, ou seja, seus resultados estão mais próximos da média dos valores e também apresenta um erro médio quadrático menor. A seguir, iremos plotá-la:

In [ ]:
#utilizando a função plot tree
plt.rc('figure', figsize=(20, 20))
tp = tree.plot_tree(arvore_2, #indica o nome da árvore
                    feature_names=X.columns,  #indica as colunas
                    filled=True) #opção estética